# Figure 4

In [ ]:
import itertools
import json
import os
import pickle
from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from statannot import add_stat_annotation

from sylseq_paper.plotting import default_plot_settings, ucsf_sequential_color_palette as colors, fancy_location_colors, smoothed_weighted_histogram
from sylseq_paper.statistics import fdr_omitnans, correlation_permutation

from matplotlib_venn import venn3

default_plot_settings(font='Helvetica', fontsize=14)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
rt_subjects = ['EC217', 'EC219', 'EC223', 'EC237', 
               'EC240', 'EC241', 'EC253', 'EC254', 
               'EC260', 'EC263', 'EC267', 'EC276']

# Set up paths
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
img_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'imaging'))
mni_img_path = os.path.join(img_dir, 'MNI_{}_{}_brain_2D.png')

sig_thresh = 0.05

boxplot_kwargs = {}
for key in ['cap', 'whisker', 'flier', 'median', 'mean']:
    boxplot_kwargs[f'{key}props'] = dict(clip_on=False)
boxplot_kwargs['boxprops'] = dict(alpha=0.75, clip_on=False)

exclude_areas = ['superiorparietal', 'inferiortemporal', 'paracentral', 'medial']

# Renaming `SMA` label to `medial SFG`
renamed_areas = {
    'location': {
        'supplementarymotor': 'medialsuperiorfrontal'
    },
    'fancy_location': {
        'SMA': 'medial SFG'
    }
}

def rename_df_areas(df):
    for label_type, renames in renamed_areas.items():
        for key, val in renames.items():
            df[label_type] = np.where(df[label_type].values == key, val, df[label_type].values)
    return df


rt_dates = {
    'target_presentation': {
        'window_type': 'target_presentation',
        'period': 'target_pres',
        'title': 'Encoding'
    },
    'delay': {
        'window_type': 'delay',
        'period': 'delay',
        'title': 'Delay'
    },
    'pre_execution': {
        'window_type': 'pre_exec',
        'period': 'pre_exec',
        'title': 'Pre-speech'
    }
}
phase_alignments = {
    'target_pres': {
        'window': [0.5, 1.5],
        'alignment': 'target_pres',
        'title': 'Target pres.'
    },
    'delay'      : {
        'window': [0.0, 0.75],
        'alignment': 'delay',
        'title': 'Delay'
    },
    'pre_exec'   : {
        'window': [-0.85, -0.1],
        'alignment': 'pre_exec',
        'title': 'Pre-speech'
    }
}

## Load RT data

In [ ]:
# Load reaction time decoding data frame
full_pdf = pd.read_hdf(os.path.join(data_dir, 'fig4_full_pdf.h5'))

# Exclude EC276 pre-speech bc reaction time dist. is so much greater than other pts. See Table S1.
pdf = full_pdf.loc[~((full_pdf.subject == 'EC276') & (full_pdf.period == 'pre_execution'))]

## Load other data (seq, cluster, AKT)

In [ ]:
# Load sequence and articulatory (syl) complexity df
seq_vs_syl_df = pd.read_hdf(os.path.join(data_dir, 'fig3_seq_vs_syl_df.h5'))

# Exclude EC276 pre-speech bc reaction time dist. is so much greater than other pts. See Table S1.
seq_vs_syl_df = seq_vs_syl_df.loc[~((seq_vs_syl_df.subject == 'EC276') & (seq_vs_syl_df.alignment == 'pre_exec'))]

seq_vs_syl_df = seq_vs_syl_df.loc[seq_vs_syl_df.subject.isin(rt_subjects)]
seq_vs_syl_df['subject_electrode'] = [f'{s}_{e}' for s, e in
                                      zip(seq_vs_syl_df.subject.values,
                                          seq_vs_syl_df.electrode.values)]
seq_vs_syl_df = seq_vs_syl_df.loc[seq_vs_syl_df.subject_electrode.isin(pdf.subject_electrode.unique())]
    
# Load dataframe electrode information
with open(os.path.join(data_dir, 'fig1_ndf_sustained_se.pkl'), 'rb') as f:
    results = pickle.load(f)
    neural_df = results['neural_df']
    sustained_se = results['sustained_se']
    
# Rename ROIs
neural_df = rename_df_areas(neural_df)
ndf = neural_df.loc[neural_df.significant_above_baseline_anytime]

pre_exec_seq_se = seq_vs_syl_df.loc[(seq_vs_syl_df.alignment.isin(['pre_exec'])) & 
                  (seq_vs_syl_df.feature_type == 'sequence') & 
                  (seq_vs_syl_df.fdr_pval < sig_thresh)].subject_electrode.values
rt_seq_sus_se = np.intersect1d(pre_exec_seq_se, sustained_se)

# Load AKT data
akt_model = pd.read_hdf(os.path.join(data_dir, 'seqsyl_AKT_model_table.h5'))
akt_se = akt_model.loc[akt_model.AKT_r > 0.1].subject_electrode.values

## Load electrode information

In [ ]:
# Compile electrode information
edf = pd.read_hdf(os.path.join(data_dir, 'all_anatomical_info.h5'))

edf = edf.loc[(~edf.location.isin(exclude_areas)) & (edf.subject.isin(rt_subjects))]
assert np.isin(rt_subjects, edf.subject.unique()).all()

# Figure panels

## RT bar chart

In [ ]:
location_lists = {
    'vSMC': ['ventralprecentral', 'ventralpostcentral'],
    'mPrCG': ['middleprecentral'],
    'Other': None
}

rt_location_colors = {
    'vSMC': fancy_location_colors['ventral PoCG'],
    'mPrCG': fancy_location_colors['middle PrCG'],
    'Other': fancy_location_colors['SMG'],
}

sig_elec_counts = {}

for location_group, cur_locations in location_lists.items():
    
    sig_elec_counts[location_group] = {key: [] for key in ['seq', 'other']}
    
    for period in rt_dates.keys():
        
        period_seq_se = seq_vs_syl_df.loc[(seq_vs_syl_df.alignment == rt_dates[period]['period']) & 
                                          (seq_vs_syl_df.feature_type == 'sequence') & 
                                          (seq_vs_syl_df.fdr_pval < sig_thresh)].subject_electrode.values
        period_seq_se = np.intersect1d(period_seq_se, sustained_se)
        
        if cur_locations is not None:
            loc_cond = pdf.location.isin(cur_locations)
        else:
            loc_cond = ~pdf.location.isin(['ventralprecentral', 'ventralpostcentral', 'middleprecentral'])
            
        all_area_elecs = pdf.loc[(pdf.period == period) & (loc_cond) & (pdf.sig_above_baseline)].shape[0]
        cur_pdf = pdf.loc[(pdf.period == period) & 
                          (pdf.significant) & 
                          (pdf.correlation > 0) & 
                          (loc_cond) &
                          (~pdf.subject_electrode.isin(akt_se))]
        seq_specific = cur_pdf.loc[cur_pdf.subject_electrode.isin(period_seq_se)].shape[0]
        not_specific = cur_pdf.shape[0] - seq_specific
        
        sig_elec_counts[location_group]['seq'].append(100 * seq_specific / all_area_elecs)
        sig_elec_counts[location_group]['other'].append(100 * not_specific / all_area_elecs)

In [ ]:
def rt_bar_chart(ax=None, fs=10, return_fig=False):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(4, 5))

    x = np.arange(len(rt_dates.keys()))
    width = 0.15

    for adjustment, location in zip([-0.2, 0, 0.2], rt_location_colors.keys()):

        ax.bar(x + adjustment, sig_elec_counts[location]['other'], width, label=location,
               bottom=sig_elec_counts[location]['seq'], color=rt_location_colors[location], alpha=0.5, edgecolor='k')

        ax.bar(x + adjustment, sig_elec_counts[location]['seq'], width, 
               edgecolor='#404040', facecolor=rt_location_colors[location], hatch='///', alpha=0.7)

    xticklabels = [rt_dates[l]['title'] for l in rt_dates.keys()]
    ax.axes.set(xticks=range(len(xticklabels)), xticklabels=xticklabels, 
                ylabel='Percent significant electrodes', ylim=(0, 25))

    handles, labels = ax.get_legend_handles_labels()
    handles.append(mpatches.Patch(facecolor='white', edgecolor='#404040', hatch='/////', alpha=0.99))
    labels.append('Sustained &\nsequence \ncomplexity')

    ax.legend(handles, labels, frameon=False, loc='upper left', fontsize=fs);
    
    for item in [ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels():
        item.set_fontsize(fs)
        
    if return_fig:
        return fig, ax
    else:
        return ax

In [ ]:
f, _ = rt_bar_chart(return_fig=True, fs=16);

## RT brain recon

In [ ]:
def rt_brain_recon(ax=None, view=None, hemi='lh', filter_by_seq_sus=True, fig=None, 
                   plot_colorbar=True, title=None, plot_legend=False, s=30):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))

    period = 'pre_execution'

    marker = 'o'

    brain_img = plt.imread(mni_img_path.format(hemi, view))
    ax.imshow(brain_img, alpha=0.5)
    ax.axis('off');
    ax.axes.set(title=title)
    
    nonsig_pdf = pdf.loc[(pdf.view == view) & 
                          (pdf.hemisphere == hemi) & 
                          (~pdf.significant) & 
                          (pdf.sig_above_baseline) &
                          (pdf.period == period)]
    ax.scatter(nonsig_pdf.warp_x, nonsig_pdf.warp_y, color='k', alpha=0.2, s=s//4)
    
    cur_pdf = pdf.loc[(pdf.view == view) & 
                      (pdf.hemisphere == hemi) & 
                      (pdf.significant) & 
                      (pdf.sig_above_baseline) &
                      (pdf.correlation > 0) &
                      (pdf.period == period)]
    
    cur_pdf = cur_pdf.loc[~cur_pdf.subject_electrode.isin(akt_se)]
    
    if filter_by_seq_sus:
        seq_marker = '^'
    else:
        seq_marker = marker
        
    colormap = mpl.colors.LinearSegmentedColormap.from_list('cmap', ['white', colors[4]])
    colormap = 'Purples'
        
    rest_pdf = cur_pdf.loc[~cur_pdf.subject_electrode.isin(rt_seq_sus_se)]
    im = ax.scatter(rest_pdf.warp_x, rest_pdf.warp_y, c=rest_pdf.correlation.values, 
                    marker=marker, s=s,
                    cmap=colormap, vmin=0.15, vmax=0.3, alpha=0.6)
    
    seq_pdf = cur_pdf.loc[cur_pdf.subject_electrode.isin(rt_seq_sus_se)]
    
    im = ax.scatter(seq_pdf.warp_x, seq_pdf.warp_y, c=seq_pdf.correlation.values, 
                    marker=seq_marker, s=s*2, ec='k',
                    cmap=colormap, vmin=0.1, vmax=0.4, alpha=0.6)
    
    if plot_colorbar:
        fig.colorbar(im, ax=ax, label='Predicted RT correlation');
        
    if filter_by_seq_sus and plot_legend:
        legend_elements = [mpl.lines.Line2D([0], [0], marker='^', color='None', label='Sustained &\nsequence complexity',
                          markerfacecolor='None', markeredgecolor='k', markersize=s//2)]
        ax.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)
        
    return ax

In [ ]:
rt_brain_recon(view='medial', plot_colorbar=False, plot_legend=True, s=20);
rt_brain_recon(view='lateral', title='Pre-speech', s=20);

## Sustained, sequence complexity, and RT overlap

In [ ]:
# All sequence complexity pre-exec elecs
all_seq_se = np.copy(pre_exec_seq_se)
        
# Find all RT sig elecs
pre_exec_rt_se = pdf.loc[(pdf.significant) & 
                (pdf.correlation > 0) & 
                (~pdf.subject_electrode.isin(akt_se)) &
                (pdf.period == 'pre_execution')].subject_electrode.values

rt_subj_sustained_se = []
for se in sustained_se:
    if not se.startswith('EC276'):
        rt_subj_sustained_se.append(se)
rt_subj_sustained_se = np.array(rt_subj_sustained_se)

In [ ]:
def se_rt_venn(ax=None, group=None, group_title=None, fs=12, complexity_feature='sequence', plot_labels=True):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
        
    if complexity_feature == 'sequence':
        set2_label = 'Sequence'
    elif complexity_feature == 'syllable':
        set2_label = 'Artic.'
    
    if group == 'all':
        set1 = set(rt_subj_sustained_se)
        
        if complexity_feature == 'sequence':
            set2 = set(all_seq_se)
        
        set3 = set(pre_exec_rt_se)
        
    else:
        # Group subject electrodes
        if group == 'mprcg':
            group_se = edf.loc[edf.location == 'middleprecentral'].subject_electrode.values
        elif group == 'ifg':
            group_se = edf.loc[edf.location.isin(['parstriangularis', 'parsopercularis'])].subject_electrode.values
        elif group == 'vsmc':
            group_se = edf.loc[edf.location.isin(['ventralprecentral', 'ventralpostcentral'])].subject_electrode.values
        else:
            raise ValueError
        
        set1 = set(np.intersect1d(rt_subj_sustained_se, group_se))
        
        if complexity_feature == 'sequence':
            set2 = set(np.intersect1d(all_seq_se, group_se))
        
        set3 = set(np.intersect1d(pre_exec_rt_se, group_se))
        
    v = venn3([set1, set2, set3], ('Sustained', f'{set2_label}\ncomplexity', 'Reaction\ntime'), ax=ax);
    
    if complexity_feature == 'sequence':
        patch_colors = {
            '100': '#F48024',
            '110': '#91856f',
            '010': '#178CCB',
            '011': '#5a76b8',
            '001': '#716FB2',
            '101': '#b2786b',
            '111': '#6e8e10'
        }
    
    for patch_id, patch_color in patch_colors.items():
        try:
            v.get_patch_by_id(patch_id).set_color(patch_color)
        except AttributeError:
            continue

    for patch in ['100', '110', '010', '011', '001', '101', '111']:
        try:
            v.get_patch_by_id(patch).set_alpha(0.5)
        except AttributeError:
            continue
        
    ax.set_title(group_title, fontsize=fs)
    
    for text in v.set_labels:
        text.set_fontsize(fs)
        if not plot_labels:
            text.set_visible(False)
        
    for text in v.subset_labels:
        if text is None:
            continue
        text.set_fontsize(fs)
    
    return ax

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(10, 5))

for cur_ax, group in enumerate(['all', 'mprcg', 'vsmc']):
    ax = se_rt_venn(group=group, group_title=group, ax=axs[cur_ax]);

## Task phases

In [ ]:
def speech_phases(axs=None, label_fontsize=12):

    if axs is None:
        fig, axs = plt.subplots(1, 2, figsize=(7, 2), gridspec_kw={'width_ratios': [3.5, 1.5]})

    line_kwargs = dict(linestyle='--', color='#404040', clip_on=False)
    
    phase_fontsize = label_fontsize

    ax = axs[0]
    ax.axvline(x=0, **line_kwargs)
    ax.annotate('Target\npres.', (0.0, 1), ha='center', va='bottom')
    ax.axvline(x=2.5, **line_kwargs)
    ax.annotate('Fixation\nCross', (2.5, 1), ha='center', va='bottom')
    ax.axes.set(xlim=(0, 3.5), xlabel='Time (s)')

    
    ax.axvspan(*phase_alignments['target_pres']['window'], 
               fc='k', alpha=0.1, zorder=0, ec='None')
    ax.axvspan(*(np.array(phase_alignments['delay']['window']) + 2.5), 
                   fc='k', alpha=0.1, zorder=0, ec='None')

    ax.annotate('Encode', (0.55, 0.05), ha='left', va='bottom', fontsize=phase_fontsize)
    ax.annotate('Delay', (2.55, 0.05), ha='left', va='bottom', fontsize=phase_fontsize)

    ax = axs[1]
    ax.axvline(x=0, **line_kwargs)
    ax.annotate('Speech\nonset', (0.0, 1), ha='center', va='bottom')
    ax.axes.set(xlim=(-1, 0.5), xlabel='Time (s)')
    ax.axvspan(*phase_alignments['pre_exec']['window'], 
                   fc='k', alpha=0.1, zorder=0, ec='None')
    ax.annotate('Pre-\nspeech', (-0.84, 0.05), ha='left', va='bottom', fontsize=phase_fontsize)

    for ax in axs:
        ax.spines['left'].set_visible(False)
        ax.yaxis.set_tick_params(length=0)
        ax.axes.set(yticks=[], ylim=(0, 1))
        ax.xaxis.set_major_locator(MultipleLocator(1))
        ax.xaxis.set_minor_locator(MultipleLocator(0.5))
        
    return axs

In [ ]:
speech_phases();

In [ ]:
# Load trials for diagram
sr = 100
ecog_trials = np.load(os.path.join(data_dir, 'fig4_hga_schematic_trials.npy'))

In [ ]:
def rt_hga(ax=None):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 2))

    for cur_trial, ecog_trial in enumerate(ecog_trials):

        ecog_trial = 0.5 * ecog_trial / np.max(np.abs(ecog_trial))
        ecog_trial -= np.mean(ecog_trial)

        ax.plot(cur_trial+ecog_trial, color='k', alpha=0.7)

    ax.axvspan(25, 100, fc='k', alpha=0.1, zorder=0, ec='None')

    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.yaxis.set_tick_params(length=0)
    ax.axes.set(yticks=[0, 0.4, 0.5, 0.6, 1, 1.35], xticks=[])
    ax.set_yticklabels(['trial$_{n}$', r'$\bf{.}$', r'$\bf{.}$', r'$\bf{.}$', 'trial$_{1}$', 'HGA'], ha='center');
    ax.tick_params(axis='y', which='major', pad=5);
    
    return ax

In [ ]:
rt_hga();

## Overlap density

In [ ]:
with open(os.path.join(data_dir, 'rt_seq_overlap_density_dict.pkl'), 'rb') as f:
    overlap_dict = pickle.load(f)

In [ ]:
def overlap_density(ax=None, feat_type=None, cmap=None, return_fig=False):
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
        
    img = plt.imread(mni_img_path.format('lh', 'lateral'))
    ax.imshow(img, alpha=0.5, zorder=0)
    ax.axis('off');

    view = 'lateral'
    hemi = 'lh'
    
    cur_df = overlap_dict['cur_df']
    in_seq = overlap_dict['in_seq']
    in_rt = overlap_dict['in_rt']

    if type(feat_type) != list:
        feat_type = [feat_type]
    
    for ft in feat_type:

        if ft == 'sequence':
            weight_list = [in_seq, in_rt]
        else:
            raise ValueError

        dens = []
        for w in weight_list:
            overlap_density, xedges, yedges = smoothed_weighted_histogram(x=cur_df.x.values, 
                                                                          y=cur_df.y.values,
                                                                          weights=w, 
                                                                          xlim=[0, img.shape[1]], 
                                                                          ylim=[0, img.shape[0]],
                                                                          bins=100,
                                                                          smooth=2,
                                                                          baseline_norm=True)
            overlap_density /= overlap_density.sum()
            dens.append(overlap_density)
        dens = np.stack(dens, axis=-1)

        dens = dens.sum(-1)
        dens /= dens.max()
        overlap_density = np.copy(dens)

        if cmap is None:
            if ft == 'sequence':
                cmap = mpl.colors.LinearSegmentedColormap.from_list('cmap', ['white', colors[0]])
            elif ft == 'syllable':
                cmap = mpl.colors.LinearSegmentedColormap.from_list('cmap', ['white', colors[3]])

        alphas = np.concatenate([np.zeros(15), np.linspace(0, 0.7, 55), 0.7*np.ones(30)])
        dx = 0.05
        for i in np.arange(0, 1, dx):
            idx = np.where(~np.logical_and(overlap_density > i, overlap_density <= i + dx))
            temp = np.copy(overlap_density)
            temp[idx] = np.nan
            ax.pcolormesh(xedges, yedges, temp.T, alpha=alphas[int(100*i)], vmin=0, vmax=1, cmap=cmap, zorder=0, rasterized=True)
    
    if return_fig:
        return fig, ax
    else:
        return ax

In [ ]:
f, _ = overlap_density(return_fig=True, feat_type=['sequence'], cmap=mpl.colors.LinearSegmentedColormap.from_list('cmap', ['white', colors[2]]))

# Overall figure

In [ ]:
##### Figure setup #####
default_plot_settings(font='Helvetica', fontsize=5, linewidth=0.5, ticklength=2)
label_fontsize = 5

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['axes.labelpad'] = 1
plt.rcParams['patch.linewidth'] = 0.25
plt.rcParams['hatch.linewidth'] = 0.25
plt.rcParams['lines.markeredgewidth'] = 0.25
plt.rcParams['axes.titlepad'] = 1
plt.rcParams['xtick.major.pad'] = 2
plt.rcParams['ytick.major.pad'] = 2
plt.rcParams['legend.handletextpad'] = 1

all_axs = {}
mm = 1/25.4
fig = plt.figure(figsize=(180*mm, 160*mm))
gs = mpl.gridspec.GridSpec(100, 100, figure=fig, wspace=50)

##########

diagram_gs = mpl.gridspec.GridSpecFromSubplotSpec(1, 2, 
                                                  subplot_spec=gs[2:13, :30], 
                                                  width_ratios=[3.5, 1.75])
all_axs['rt_diagram'] = [fig.add_subplot(diagram_gs[0, i]) for i in range(2)]
all_axs['rt_diagram'] = speech_phases(axs=all_axs['rt_diagram'], label_fontsize=label_fontsize)

all_axs['rt_hga'] = fig.add_subplot(gs[20:30, :12])
all_axs['rt_hga'] = rt_hga(ax=all_axs['rt_hga'])

all_axs['rt_barchart'] = fig.add_subplot(gs[:32, 38:55])
all_axs['rt_barchart'] = rt_bar_chart(ax=all_axs['rt_barchart'], fs=label_fontsize)

all_axs['rt_lateral'] = fig.add_subplot(gs[8:35, 57:])
all_axs['rt_lateral'] = rt_brain_recon(ax=all_axs['rt_lateral'], view='lateral', fig=fig, plot_colorbar=True, s=10)

all_axs['rt_medial'] = fig.add_subplot(gs[:10, 56:72])
all_axs['rt_medial'] = rt_brain_recon(ax=all_axs['rt_medial'], view='medial', fig=fig, plot_colorbar=False, plot_legend=True, s=10)

ax = fig.add_subplot(gs[:6, 50:])
ax.axis('off')
ax.axes.set(title='Pre-speech')

all_axs['venn_all'] = fig.add_subplot(gs[30:75, :20])
all_axs['venn_all'] = se_rt_venn(ax=all_axs['venn_all'], group='all', group_title='All', fs=label_fontsize)

all_axs['venn_mprcg'] = fig.add_subplot(gs[30:75, 22:42])
all_axs['venn_mprcg'] = se_rt_venn(ax=all_axs['venn_mprcg'], group='mprcg', group_title='mPrCG', plot_labels=False, fs=label_fontsize)

all_axs['venn_vsmc'] = fig.add_subplot(gs[30:75, 40:60])
all_axs['venn_vsmc'] = se_rt_venn(ax=all_axs['venn_vsmc'], group='vsmc', group_title='vSMC', plot_labels=False, fs=label_fontsize)

all_axs['density'] = fig.add_subplot(gs[40:67, 57:95])
all_axs['density'] = overlap_density(ax=all_axs['density'], feat_type=['sequence'], 
                                     cmap=mpl.colors.LinearSegmentedColormap.from_list('cmap', ['white', colors[2]]))


##########
panel_axs = {}
panel_axs['a'] = fig.add_subplot(gs[:5, :15])
panel_axs['b'] = fig.add_subplot(gs[:5, 40:50])
panel_axs['c'] = fig.add_subplot(gs[:5, 62:100])
panel_axs['d'] = fig.add_subplot(gs[42:47, :30])
panel_axs['e'] = fig.add_subplot(gs[42:47, 65:])

for panel, ax in panel_axs.items():
    ax.axis('off')
    ax.annotate(panel, (-20, 25), xycoords='axes points', ha='right', fontsize=7, weight='bold');